In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, VECM
from statsmodels.tsa.vector_ar.vecm import select_order, select_coint_rank, coint_johansen
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import norm

import utils

# Set global options
pd.options.display.float_format = '{:.10f}'.format
sns.set_theme(style="whitegrid")

In [ ]:
df = pd.read_csv('data/prices.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace = True)
sample_df = df.iloc[:575]
df.head()
sample_df.tail()
sample_df.info()

In [ ]:
coint_df = sample_df[['y3', 'y8']]
coint_df.head()


In [ ]:
# print(coint_df.dtypes)
print(coint_df.index)
coint_df = coint_df.dropna().astype(float)

Let's again see how both series looks on one diagram

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(coint_df['y3'], label='Y3', color='black')
plt.plot(coint_df['y8'], label='Y10', color='blue')
plt.title('Cointegrated series')
plt.xlabel('Date')
plt.ylabel('Asset Price Index')
plt.legend(loc='upper left')
plt.grid(True, which='major', axis='x', linestyle='--')
plt.show()

## Cointegration Test ##

 ΔYt = ΠYt−1 + Γ1ΔYt−1 + Γ2ΔYt−2 + . . . + Γp−1ΔYt−(p−1) + εt

Because lag length can affect Johansen test we should first decide how many lags will be taken into consideration. First what comes to mind is 5 lags because we are dealing with asset prices, it would make sense. But to make more informed decision we can employ information criteria.

In [ ]:
model = VAR(coint_df,freq='D')
results = model.select_order(maxlags=10)
print("\nLag selection results:")
print(results.summary())

We are looking at lowest values for each criteria. We can see that two of the information criteria are pointing to 3 lags and two to 4. Let's use method for picking up the most optimal number of lags.

In [ ]:

utils.selct_var_order(coint_df, max_lags=10)

Given that we are dealing with financial instruments data we will use 3 number of lags in our test. Also AIC and FPE usually favor bigger number of lags, BIC is usually on the lower side, but HQIC should be something in between.

# Performing Johansen test

We are performing this test to verify if cointegrated vector exists for those two time series.

In [ ]:
# Perform Johansen test
# K=4 in levels VAR -> k_ar_diff = K-1 = 3 lags in VECM differences
# ecdet = "const" -> det_order = 0 (constant in CE)

johansen_result = coint_johansen(sample_df[['y3', 'y8']], det_order=0, k_ar_diff=3)

print("Johansen Test Results:")
print("Eigenvalues:")
print(johansen_result.eig)
print("\nTrace Statistic:")
print(johansen_result.lr1)
print("\nCritical Values (90%, 95%, 99%) for Trace Statistic:")
print(johansen_result.cvt)
print("\nMaximum Eigenvalue Statistic:")
print(johansen_result.lr2)
print("\nCritical Values (90%, 95%, 99%) for Max Eigenvalue Statistic:")
print(johansen_result.cvm)

In [ ]:
print("--- Interpretation (Trace Test) ---")
hypotheses_trace = ['r <= 0', 'r <= 1']
for i in range(len(hypotheses_trace)):
    print(f"H0: {hypotheses_trace[i]}")
    print(f"  Trace Statistic: {johansen_result.lr1[i]:.3f}")
    print(f"  Critical Value (95%): {johansen_result.cvt[i, 2]:.3f}")
    if johansen_result.lr1[i] > johansen_result.cvt[i, 2]:
        print("  Result: Reject H0 at 5% significance level.")
    else:
        print("  Result: Cannot reject H0 at 5% significance level.")

print("\n--- Interpretation (Max Eigenvalue Test) ---")
hypotheses_maxeig = ['r = 0', 'r = 1'] # H0: rank is r vs H1: rank is r+1
for i in range(len(hypotheses_maxeig)):
    print(f"H0: {hypotheses_maxeig[i]}")
    print(f"  Max Eigenvalue Statistic: {johansen_result.lr2[i]:.3f}")
    print(f"  Critical Value (95%): {johansen_result.cvm[i, 2]:.3f}")
    if johansen_result.lr2[i] > johansen_result.cvm[i, 2]:
        print("  Result: Reject H0 at 5% significance level.")
    else:
        print("  Result: Cannot reject H0 at 5% significance level.")

In both cases for Trace and Max Eigenvalue tests we can see that H0 indicating that there is **no cointegrating vector can be rejected**. However, we fail to reject H0 that there is **one cointegrating vector** thus implying that we have exactly one cointegrating vector and series is cointegrated.

## VECM model ##

Following variables will be used in our VECM model
* Previously set number of lags = 4 so lag order in differences will be 3.
* Cointegration as already establish is of rank one
* Daily frequency = daily
* why ci exactly?

In [ ]:
# Estimate VECM
vecm_model = VECM(coint_df, k_ar_diff=3, coint_rank=1, deterministic='ci',freq='D')
vecm_results = vecm_model.fit()

Let's see the results:

In [ ]:
print(vecm_results.summary())

The VECM is given by:

$$\Delta\boldsymbol{y}_t = \Pi\boldsymbol{y}_{t-1} + \Gamma_1\Delta\boldsymbol{y}_{t-1} + ... + \Gamma_p\Delta\boldsymbol{y}_{t-p} + \boldsymbol{c}_d + \boldsymbol{\varepsilon}_t$$

where long run coefficient matrix is described as:
$$
\boldsymbol{\Pi}=\alpha\beta'=\left[
	\begin{array}{ccc}
	\alpha_{11} \\
	\alpha_{21} \\
	\end{array}
\right]
\left[
	\begin{array}{cccc}
\beta_{11} \quad \beta_{21} \\
 \end{array}
\right]
$$


let's see what values it takes in our case:

In [ ]:
print("\nLong-run coefficient matrix:")
print(f"{vecm_results.alpha} {vecm_results.beta[:, 0] / vecm_results.beta[0, 0]}")

**Interpretations:**

Alphas ( 'adjustment parameters' )
* $\alpha_{11}$ - we see it being positive, and in this case we would expect negative sign, but coefficient is not statistically significant.
* $\alpha_{12}$ - is positive 2.5163 and statistically significant, meaning that correction mechanism works in expected direction. Variable should return to the long-term equilibrium - when it's above it should adjust downwards and below it adjust upwards.

 Betas ( coingegrating vectors )
* $\beta_{11}$ - normalization applied thanks to decomposition of Π no being unique.
* $\beta_{12}$ - negative sign tells us that y$_{1}$ it is positively related to y$_{2}$. In perfect equilibrium we would have y$_{1}$ = 0.625 * y$_{2}$.

Lag terms:
$$\Gamma_1\Delta\boldsymbol{y}_{t-1} + ... + \Gamma_p\Delta\boldsymbol{y}_{t-p} + \boldsymbol{c}_d + \boldsymbol{\varepsilon}_t$$

this represents short-run dynamics outside cointegration relation. None of the coefficients is statistically significant thus we are ommiting interpretation.

**We are reparametrizing VECM to VAR for further examination of the relations**

In [ ]:
# Estimate VAR
var_model = VAR(coint_df,freq='D')
var_results = var_model.fit(4)

## Impulse Response Functions
Let's examine what effect have shocks applied to variables

In [ ]:
# Calculate and plot impulse response functions
irf = var_results.irf(160)  # 36 periods ahead (equivalent to n.ahead=36 in R)
# and hence interpretations may change depending on variable ordering.
irf.plot(orth=True);

**Interpretation**

* y3->y3 : initial increase up to 25 days, and afterward slow decrease toward equilibrium
* y3->y8 : shock has bigger initial impact on y8, but as in previous case impacted variable is slowly converging toward 0
* y8->y3 : no first period impact of the y8 on y3. But we can observe negative impact on y3 and which converges back to 0
* y8->y8 : quick small jump at the beginning but as in previous case we are observing negative impact on first periods and convergence starting prior 25 period.


